# IMPORTAÇÃO DOS PACOTES NECESSARIOS

In [ ]:
from keras.utils import HDF5Matrix
from keras.utils import np_utils
import keras.models 
import numpy as np
from keras.models import Sequential
from keras.models import model_from_json
import tensorflow as tf
from keras.layers import Dense
import matplotlib as plt
from keras.utils import np_utils
import os
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Input, Flatten,Conv2D, MaxPooling2D,Dropout,BatchNormalization,Embedding
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# IMPORTANDO OS PESOS VGG16 'IMAGENET'

In [ ]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
vgg_conv.summary()

In [ ]:
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

In [ ]:
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

# ADCIONANDO CAMADAS NA NOVA REDE NEURAL COM DUAS SAIDAS

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

# APLICANDO ALGUMAS ALTERAÇOES NA DIMENSÃ DA IMAGEM

In [ ]:
from keras.preprocessing.image import ImageDataGenerator



train_dir = 'C:\\Users\\Renesio\\Documents\\TCC2\\tudo\\'
validation_dir ='C:\\Users\\Renesio\\Documents\\TCC2\\validacao'


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
type(train_datagen)

# TREINANDO O NOVO MODELO

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')